In [2]:
# Import the libraries 
import nltk 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
import numpy as np 
import gensim
from gensim.models import Word2Vec
import pandas as pd 

In [14]:
# Load the dataset 
df_data = pd.read_csv("E:\Tarang\Ashoka\Python\PYTHON PROJECT\PM_Modi_Speech_Text_english.csv")

In [34]:
df_data

,Unnamed: 0,Date,Speech,Title
0,2,09 JAN 2017,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...
1,3,09 JAN 2017,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...
2,5,03 JAN 2017,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...
3,6,26 FEB 2017,"My dear countrymen, Namaskar. Winter is on its...",English Translation of the text of PM’s ‘Mann ...
4,8,21 FEB 2017,Namaskaram. Greetings to everyone. Swami Nirvi...,Text of PM’s inaugural address (via video conf...
...,...,...,...,...
1177,1263,08 APR 2023,Bharat Mata Ki Jai!\n\nBharat Mata Ki Jai!\n\n...,English rendering of PM’s address at launch of...
1178,1264,04 APR 2023,ExcellenciesHonourable Minister Mr. Harbers;Sp...,"Text of Address by Dr. P.K. Mishra, Principal ..."
1179,1265,04 APR 2023,"Namaskar!Excellencies, heads of state, Academi...",Text of PM’s remarks at International Conferen...
1180,1266,03 APR 2023,My colleague in the Union Cabinet Dr. Jitendra...,English rendering of PM’S address at the diamo...


In [35]:
# Converts the Speech column to a string data type.
df_data['Speech'] = df_data['Speech'].astype(str)

In [36]:
# Initializing the model 
lemmatizer = WordNetLemmatizer()

In [37]:
def cleaned(speech):
    # Sentence Tokenization
    sentence = nltk.sent_tokenize(speech)

    # Creating an empty list to store the cleaned dataset 
    corpus = []

    # Loop through each sentence in the speech
    for i in range(len(sentence)):
        # Remove any non-alphabetic characters in the sentence and convert to lowercase
        review = re.sub("[^a-zA-Z0-9]", " ", sentence[i])
        review = review.lower()
        
        # Tokenize the sentence into a list of words
        review = review.split()
        
        # Lemmatize each word in the sentence and remove stop words
        lemmatizer = WordNetLemmatizer()
        review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
        
        # Convert the list of words back into a sentence
        review = ' '.join(review)
        
        # Add the cleaned sentence to the corpus
        corpus.append(review)

    return corpus[:-1] # Removing the last element of the list 

In [40]:
def vectorize(corpus):

    # Tokenize each sentence in the corpus into a list of words
    corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in corpus]

    # Train the Word2Vec model on the tokenized corpus
    model = Word2Vec(sentences=corpus_tokenized, vector_size=300, window=5, min_count=1, workers=4)

    # Extract the word vectors for each word in the corpus
    vectors = []
    for sentence in corpus_tokenized:
        for word in sentence:
            if word in model.wv.key_to_index:
                # If the word is in the Word2Vec model's vocabulary, extract its vector
                word_vector = model.wv.get_vector(word)
                vectors.append(np.average(word_vector))
            else:
                # If the word is not in the vocabulary, print a message to the console
                print(f"{word} not in vocabulary.") 
    
    return vectors   


In [39]:
# To convert cleaned speeches into string for performing NER
def string(speech):
    string_speech = " ".join(speech)
    return string_speech

In [43]:
df_data.to_csv("E:\Tarang\Ashoka\Python\PYTHON PROJECT\PM_Modi_Speech_Cleaned_string.csv")

In [41]:
# Clean the text data in the 'Speech' column of the dataframe using the 'cleaned' function
# and create a new column 'Cleaned_Speech' to store the cleaned text data
df_data['Cleaned_Speech'] = df_data['Speech'].apply(cleaned)

# Convert the cleaned text data in the 'Cleaned_Speech' column to a string 
# using the 'string' function and create a new column 'Cleaned_Speech_string'
# to store the string data
df_data['Cleaned_Speech_string'] = df_data['Cleaned_Speech'].apply(string)

# Vectorize the cleaned text data in the 'Cleaned_Speech' column
# using the 'vectorize' function and create a new column 'Vectorized_Speech'
# to store the vectorized data
df_data['Vectorized_Speech'] = df_data['Cleaned_Speech'].apply(vectorize)

RuntimeError: you must first build vocabulary before training the model

In [ ]:
df_data['Vectorized_Speech'] = df_data['Cleaned_Speech'].apply(vectorize)

In [45]:
matrix = [] 
for i in range(df_data.shape[0]):
    matrix.append(df_data['Vectorized_Speech'][i])

In [48]:
np_m = np.array(matrix)

In [67]:
np_m_df = pd.DataFrame(np_m)
np_m_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.000055,0.000329,0.000114,0.000119,-0.000108,-0.000408,0.000313,0.000755,0.000032,-0.000158,...,-0.000012,0.000391,0.000274,-0.000001,0.000454,0.000449,0.000120,-0.000147,0.000272,-0.000329
1,-0.000020,0.000442,0.000112,0.000208,0.000005,-0.000525,0.000422,0.001080,-0.000090,-0.000192,...,0.000161,0.000536,0.000385,-0.000027,0.000712,0.000668,0.000166,-0.000313,0.000258,-0.000369
2,-0.000026,0.000505,-0.000108,0.000167,-0.000011,-0.000569,0.000331,0.000908,0.000137,-0.000315,...,0.000006,0.000686,0.000301,-0.000062,0.000543,0.000712,-0.000019,-0.000205,0.000307,-0.000304
3,-0.000012,0.000445,0.000062,0.000202,0.000027,-0.000410,0.000272,0.000806,0.000136,-0.000138,...,0.000021,0.000378,0.000319,0.000064,0.000324,0.000479,-0.000025,-0.000281,0.000294,-0.000148
4,-0.000045,0.000299,-0.000050,0.000097,-0.000038,-0.000325,0.000155,0.000604,0.000111,-0.000187,...,0.000020,0.000328,0.000179,-0.000012,0.000353,0.000362,-0.000007,-0.000128,0.000128,-0.000127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,-0.000030,0.000492,0.000086,0.000339,-0.000139,-0.000608,0.000412,0.001221,0.000014,-0.000419,...,0.000115,0.000690,0.000379,-0.000022,0.000653,0.000765,0.000104,-0.000259,0.000380,-0.000400
1178,0.000045,0.000287,0.000016,0.000176,0.000065,-0.000276,0.000197,0.000679,-0.000118,-0.000070,...,0.000083,0.000295,0.000242,-0.000094,0.000360,0.000451,0.000098,-0.000203,0.000176,-0.000201
1179,0.000030,0.000334,0.000005,0.000187,-0.000125,-0.000368,0.000311,0.000792,-0.000078,-0.000285,...,0.000108,0.000422,0.000147,-0.000026,0.000444,0.000437,0.000049,-0.000237,0.000317,-0.000332
1180,-0.000045,0.000286,0.000013,0.000128,-0.000006,-0.000425,0.000185,0.000757,0.000114,-0.000169,...,0.000114,0.000419,0.000290,0.000027,0.000461,0.000527,-0.000023,-0.000118,0.000197,-0.000200


In [69]:
# Min Max Scaling
np_m_df_norm = (np_m_df-np_m_df.min())/(np_m_df.max()-np_m_df.min())
np_m_df_norm

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.879742,0.085635,0.280214,0.149869,0.410721,0.964471,0.049500,0.020517,0.227270,0.895903,...,0.498700,0.027320,0.049409,0.581606,0.049421,0.030612,0.551777,0.901156,0.069885,0.832883
1,0.836504,0.106268,0.279186,0.189869,0.477076,0.941234,0.079705,0.055203,0.162551,0.879453,...,0.610892,0.057937,0.075616,0.569215,0.124347,0.073970,0.582357,0.856149,0.065231,0.815615
2,0.832703,0.117942,0.111305,0.171239,0.467508,0.932522,0.054544,0.036838,0.282884,0.820145,...,0.510437,0.089846,0.055937,0.551652,0.075254,0.082660,0.458407,0.885343,0.081385,0.843604
3,0.840986,0.106825,0.240669,0.187034,0.490078,0.964111,0.038195,0.026015,0.281973,0.905181,...,0.519874,0.024470,0.060213,0.613723,0.011912,0.036528,0.454286,0.864814,0.077277,0.910023
4,0.821982,0.080235,0.155331,0.139735,0.451723,0.980944,0.005654,0.004388,0.269059,0.881707,...,0.519372,0.013811,0.027030,0.576257,0.020236,0.013313,0.466610,0.906133,0.022629,0.918730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,0.830493,0.115558,0.259110,0.248762,0.392465,0.924909,0.076717,0.070264,0.217646,0.770051,...,0.580971,0.090674,0.074362,0.571345,0.107182,0.093079,0.540789,0.870597,0.105248,0.802517
1178,0.873879,0.078034,0.205670,0.175532,0.512358,0.990566,0.017361,0.012375,0.147338,0.938298,...,0.560101,0.006802,0.041853,0.535858,0.022217,0.030894,0.537090,0.885977,0.038622,0.887289
1179,0.865288,0.086543,0.197648,0.180101,0.400704,0.972449,0.048958,0.024447,0.168668,0.834417,...,0.576484,0.033808,0.019456,0.569702,0.046682,0.028150,0.504341,0.876764,0.084780,0.831694
1180,0.821766,0.077706,0.203304,0.153766,0.470526,0.961035,0.014116,0.020693,0.270386,0.890261,...,0.580202,0.033275,0.053359,0.595481,0.051605,0.046055,0.455575,0.908770,0.045537,0.887658


In [71]:
len(np.unique(np_m))

353510

In [33]:
df_data.to_csv("E:\Tarang\Ashoka\Python\PYTHON PROJECT\PM_Modi_Speech_Cleaned.csv")

In [20]:
df1 = pd.read_csv("E:\Tarang\Ashoka\Python\PYTHON PROJECT\PM_Modi_Speech_Cleaned.csv")

In [21]:
df1

,Unnamed: 0.1,Unnamed: 0,Date,Speech,Title,Vectorized_Speech,Cleaned_Speech
0,0,2,09 JAN 2017,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...,[ 5.50157347e-05 3.28888331e-04 1.13795533e-...,['chief minister gujarat shri vijay rupani ji ...
1,1,3,09 JAN 2017,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...,[-1.96002020e-05 4.41547512e-04 1.12444672e-...,['delighted gift city inaugurate india first i...
2,2,5,03 JAN 2017,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...,[-2.61597761e-05 5.05288888e-04 -1.08021282e-...,['governor andhra pradesh shri e l narasimhan ...
3,3,6,26 FEB 2017,"My dear countrymen, Namaskar. Winter is on its...",English Translation of the text of PM’s ‘Mann ...,[-1.18660855e-05 4.44589416e-04 6.18628110e-...,"['dear countryman namaskar', 'winter way', 'va..."
4,4,8,21 FEB 2017,Namaskaram. Greetings to everyone. Swami Nirvi...,Text of PM’s inaugural address (via video conf...,[-4.46609192e-05 2.99401523e-04 -5.02047769e-...,"['namaskaram', 'greeting everyone', 'swami nir..."
...,...,...,...,...,...,...,...
1177,1177,1263,08 APR 2023,Bharat Mata Ki Jai!\n\nBharat Mata Ki Jai!\n\n...,English rendering of PM’s address at launch of...,[-2.99734602e-05 4.92271967e-04 8.60807631e-...,"['bharat mata ki jai', 'bharat mata ki jai', '..."
1178,1178,1264,04 APR 2023,ExcellenciesHonourable Minister Mr. Harbers;Sp...,"Text of Address by Dr. P.K. Mishra, Principal ...",[ 4.48977480e-05 2.87385832e-04 1.59012397e-...,['excellencieshonourable minister mr harbers s...
1179,1179,1265,04 APR 2023,"Namaskar!Excellencies, heads of state, Academi...",Text of PM’s remarks at International Conferen...,[ 3.00730444e-05 3.33847944e-04 5.36639072e-...,['namaskar excellency head state academic busi...
1180,1180,1266,03 APR 2023,My colleague in the Union Cabinet Dr. Jitendra...,English rendering of PM’S address at the diamo...,[-4.50328298e-05 2.85595073e-04 1.27945341e-...,['colleague union cabinet dr jitendra singh ji...


In [33]:
df1['Cleaned_Speech'][321]

"['excellency friend president solih lady gentleman glad privilege visiting beautiful country maldives first foreign visit second term', 'matter even greater happiness got opportunity meet close friend like', 'behalf team thank maldives government heartfelt gratitude opportunity wonderful hospitality', 'country celebrated festival eid day ago joy glee', 'wish light festival always shine life citizen', 'excellency today honoring highest honor maldives honored whole india', 'honor distinguished rule izzudeen matter joy pride', 'honor also respect friendship close relation two country', 'behalf indian accept great humility gratitude', 'thousand year two country bound wave indian ocean close historical cultural tie', 'unwavering friendship also guide difficult time', 'whether external assault natural disaster like tsunami recent paucity drinking water', 'india always standing maldives always come forward help first', 'friend clear mandate parliamentary election india presidential majlis el

In [27]:
df1['Speech'][1000]

"Wishing all the people of Bhavnagar a very Happy Navratri! First of all, I have to apologize to Bhavnagar for coming here after such a long period. This has happened for the first time. I apologize as I could not come before. I would never forget the blessings you showered on me and the love you have given me. I can see such a large number of people far and wide and that too on a scorching day. I bow to all of you.This meeting in Bhavnagar is special today. On the one hand, the country has completed 75 years of independence while Bhavnagar is going to complete 300 years of its foundation day this year. In this journey of 300 years, Bhavnagar has made a mark for its steady growth as well as being the cultural capital of Saurashtra. Several projects worth crores of rupees have been either inaugurated or foundation stone has been laid here today to give a new dimension to this development journey. These projects will empower the identity of Bhavnagar, will give a new gift of irrigation t